In [1]:
import pysolr
import spacy
import requests
import datetime
import json
import dateutil.parser
from IPython.display import Markdown, display
nlp = spacy.load('en_core_web_lg')

### Make a request to the Mapquest service

In [2]:
#MapQuest Geocode service.  More info: https://developer.mapquest.com/documentation/geocoding-api/
mapquest_url='https://www.mapquestapi.com/geocoding/v1/address?key=FVfj6GGCXVEGUjXvokn4IbXdNAHIbbH0&location='
def geocode(text):
    req = requests.get(mapquest_url+text)
    jsn = req.json()
    loc = None
    if "results" in jsn and len(jsn["results"]) and "locations" in jsn["results"][0] and len(jsn["results"][0]["locations"]):
        loc = jsn["results"][0]["locations"][0]
    return loc

In [3]:
#Transform a Mapquest latLng point to a Solr geofilt point
def latlng_point(loc):
    latlng = loc["latLng"]
    return str(latlng["lat"]) + ',' + str(latlng["lng"])

### Get entities with SpaCy

In [4]:
def extract_chunked_locations(text):
    #Use a basic finite state machine to chunk proper noun GPEs as one location
    #Maintain positions of the tokens for removal from the query
    doc = nlp(text)
    gpes = []
    for s in doc.sents:
        curr = []
        start = None
        end = None
        last = None
        isgpe = False
        for t in s:
            if (t.pos_ == 'PROPN' and t.ent_type_ == 'GPE') or (isgpe == True and t.text==','):
                isgpe = True
                if t.text!=',':
                    curr.append(t.text)
                    if (start is None):
                        start = t.doc[t.i:t.i].start_char
                    last = t
            elif isgpe == True:
                end = last.doc[last.i:last.i].start_char + len(last)
                gpes.append({"text":' '.join(curr),"start":start,"end":end})
                curr = []
                start = None
                end = None
                isgpe = False
            else:
                isgpe = False
        if(len(curr)):
            end = last.doc[last.i:last.i].start_char + len(last)
            gpes.append({"text":' '.join(curr),"start":start,"end":end})                 
            curr = []
            start = None
            end = None
            isgpe = False
    return gpes

In [5]:
print(extract_chunked_locations("Kevin McCallister in New York NY"))
print(extract_chunked_locations("Kevin alone in NYC"))
print(extract_chunked_locations("Indiana Jones India"))
print(extract_chunked_locations("Los Angeles escape"))
print(extract_chunked_locations("boxing revenge in moscow"))
print(extract_chunked_locations("boxing revenge in Moscow"))
print(extract_chunked_locations("Riddick in the underverse"))

[{'text': 'New York NY', 'start': 21, 'end': 32}]
[{'text': 'NYC', 'start': 15, 'end': 18}]
[{'text': 'Indiana', 'start': 0, 'end': 7}, {'text': 'India', 'start': 14, 'end': 19}]
[{'text': 'Los Angeles', 'start': 0, 'end': 11}]
[]
[{'text': 'Moscow', 'start': 18, 'end': 24}]
[]


### Enrich a query with a location using SpaCy and Mapquest

In [6]:
def enrich_query_location(q):
    #q=*:*&fq={!geofilt%20sfield=location}&pt=18.36336,-66.07684&d=50&fl=title,overview,location*
    enrichment = {"q":q}
    entities = extract_chunked_locations(q)
    if len(entities):
        #found a location - look it up in mapquest and disambiguate
        entity = entities[0]
        geocoded = geocode(entity["text"])
        if geocoded:
            # It's a valid location!
            # ...add the filter query and remove the text from the query:
            latlng = latlng_point(geocoded)
            fq = "{!geofilt sfield=location}"
            pt = latlng
            d = 20
            enrichment["fq"] = fq
            enrichment["pt"] = pt
            enrichment["d"] = d
            enrichment["q"] = q[0:entity["start"]] + q[entity["end"]:]
    return enrichment

#Test it out:
enrich_query_location("Kevin alone in New York, NY")

{'q': 'Kevin alone in ',
 'fq': '{!geofilt sfield=location}',
 'pt': '40.713054,-74.007228',
 'd': 20}

In [21]:
#Solr Client
solr = pysolr.Solr('http://localhost:8983/solr/tmdb')

#Formats a search result as markdown and prints
def printmd(title,location,date,overview):
    title = '### ' + title + '\n'
    date = '_' + str(date) + '_\n'
    location = '_' + str(location) + '_\n'
    overview = '> ' + overview
    display(Markdown(title))
    display(Markdown(location))
    display(Markdown(date))
    display(Markdown(overview))

#Print the Title, Release Date, and Overview from TMDB
def printresults(res):
    for r in res:
        print('\n---')
        if "location" in r and len(r["location"]):
            printmd(r["title"][0],r["location"][0],r["release_date"],r["overview"][0])
        else:
            printmd(r["title"][0],"NO LOCATION DATA",r["release_date"],r["overview"][0])

def solrquery(q):
    if len(q):
        return "title_en:("+q+")^1.2 overview_en:("+q+")"
    else:
        return None
        
#Enrich and Search a text query
def search(query):
    enriched = enrich_query_location(query)
    q = solrquery(enriched["q"])
    print("Searching for `" + query + "` ...")
    if q:
        print("  q = " + q)
    if "fq" in enriched and isinstance(enriched["fq"], str):
        fq = enriched["fq"]
        pt = enriched["pt"]
        d = enriched["d"]
        print("  fq = " + fq)
        print("  pt = " + pt)
        print("  d = " + str(d))
        res = solr.search(q=q,fq=fq,pt=pt,d=d,fl="title,overview,location,location_city,location_state,location_country,release_date",rows=3)
    else:
        print("  ...no enrichments")
        res = solr.search(q=q,fl="title,overview,release_date",rows=3)

    return res

### Let's try it with some good examples

In [22]:
printresults(search("Kevin McCallister in New York NY"))

Searching for `Kevin McCallister in New York NY` ...
  q = title_en:(Kevin McCallister in )^1.2 overview_en:(Kevin McCallister in )
  fq = {!geofilt sfield=location}
  pt = 40.713054,-74.007228
  d = 20

---


### Home Alone 2: Lost In New York


_40.713054,-74.007228_


_1992-11-19T00:00:00Z_


> Instead of flying to Florida with his folks, Kevin ends up alone in New York, where he gets a hotel room with his dad's credit card—despite problems from a clerk and meddling bellboy. But when Kevin runs into his old nemeses, the Wet Bandits, he's determined to foil their plans to rob a toy store on Christmas eve.

In [11]:
printresults(search("Indiana Jones India"))

Searching for `Indiana Jones India` ...
  q = title_en:( Jones India)^1.2 overview_en:( Jones India)
  fq = {!geofilt sfield=location}
  pt = 39.919991,-86.2818
  d = 20

---


### Indiana Jones and the Temple of Doom


_39.919991,-86.2818_


_1984-05-23T00:00:00Z_


> After arriving in India, Indiana Jones is asked by a desperate village to find a mystical stone. He agrees – and stumbles upon a secret cult plotting a terrible plan in the catacombs of an ancient palace.

...


---


### Armour of God


_39.919991,-86.2818_


_1986-08-16T00:00:00Z_


> Jackie Chan stars as Asian Hawk, an Indiana Jones-style adventurer looking to make a fortune in exotic antiquities. After Hawk discovers a mysterious sword in Africa, a band of Satan-worshipping monks kidnap his ex-girlfriend Lorelei, demanding the sword as ransom as well as other pieces of the legendary Armour of God - a magical outfit dating back to the Crusades.

In [12]:
printresults(search("Los Angeles escape"))

Searching for `Los Angeles escape` ...
  q = title_en:( escape)^1.2 overview_en:( escape)
  fq = {!geofilt sfield=location}
  pt = 34.052238,-118.243344
  d = 20

---


### Escape from L.A.


_34.052238,-118.243344_


_1996-08-09T00:00:00Z_


> This time, a cataclysmic temblor hits Los Angeles, turning it into an island. The president views the quake as a sign from above, expels Los Angeles from the country and makes it a penal colony for those found guilty of moral crimes. When his daughter, part of a resistance movement, steals the control unit for a doomsday weapon, Snake again gets tapped to save the day.

...


---


### Blade Runner


_34.052238,-118.243344_


_1982-06-25T00:00:00Z_


> In the smog-choked dystopian Los Angeles of 2019, blade runner Rick Deckard is called out of retirement to terminate a quartet of replicants who have escaped to Earth seeking their creator for a way to extend their short life spans.

In [13]:
printresults(search("boxing revenge in Moscow"))

Searching for `boxing revenge in Moscow` ...
  q = title_en:(boxing revenge in )^1.2 overview_en:(boxing revenge in )
  fq = {!geofilt sfield=location}
  pt = 55.751634,37.618704
  d = 20

---


### Rocky IV


_55.751634,37.618704_


_1985-11-21T00:00:00Z_


> Rocky must come out of retirement to battle a gargantuan Soviet fighter named Drago, who brutally punished Rocky's friend and former rival, Apollo Creed. Seeking revenge in the name of his fallen comrade and his country, Rocky agrees to fight Drago in Moscow on Christmas, and the bout changes both fighters -- and the world.

In [31]:
printresults(search("Manhattan")) #There is a movie 'Manhattan' (Woody Allen, 1979)

Searching for `Manhattan` ...
  fq = {!geofilt sfield=location}
  pt = 40.753259,-74.003804
  d = 20

---


### Sleepy Hollow


_40.713054,-74.007228_


_1999-11-18T00:00:00Z_


> New York detective Ichabod Crane is sent to Sleepy Hollow to investigate a series of mysterious deaths in which the victims are found beheaded. But the locals believe the culprit to be none other than the ghost of the legendary Headless Horseman.


---


### The April Fools


_40.713054,-74.007228_


_1969-05-23T00:00:00Z_


> Newly-promoted if none too happily married Howard Brubaker leaves a rowdy Company party early with the stunning Catherine, whom it turns out is herself unhappily married - to the boss. They spend an innocent night in New York becoming more and more attracted to each other, so that when Catherine announces she intends to leave her husband and return to Paris Howard asks to go along too. In the cold light of morning problems and pressures from spouses crowd in